# **Frames Extraction**
In this section, frames are extracted from the videos .



<img src="https://i.ytimg.com/vi/-lBArWkwcBk/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLDd2c8c16x21TnFHc1gcyNiUlypag" width="500">


### Training data

In [ ]:
import cv2
import os
from glob import glob

# Base dataset directory
base_input_dir = r"\nthu_ddd\training_dataset"

# Output directory where extracted frames will be saved
output_dir = r"\nthu_ddd\nthu_ddd1"

# Get all .avi video files
video_files = glob(os.path.join(base_input_dir, "*", "*.avi"))

# Print number of videos
print(f"number of videos: {len(video_files)}")

for video_path in video_files:
    # Split path correctly for Windows
    path_parts = video_path.split("\\")
    
    person_id = path_parts[-3]        # Person ID
    scenario = path_parts[-2]         # Scenario name
    video_filename = os.path.splitext(path_parts[-1])[0]   # Video name without extension

    # Combine scenario + video name
    scenario_video_name = f"{scenario}_{video_filename}"

    # Path to the label text file
    label_txt_path = os.path.join(base_input_dir, person_id, scenario,
                                  f"{person_id}_{video_filename}_drowsiness.txt")

    # Read all labels and store them as a list
    with open(label_txt_path, "r") as f:
        labels = list(f.read().strip())

    # Output directory for this specific video
    video_output_dir = os.path.join(output_dir, person_id, scenario_video_name)

    # Two class folders
    drowsy_dir = os.path.join(video_output_dir, "drowsy")
    awake_dir = os.path.join(video_output_dir, "awake")

    # Create directories if they don't exist
    os.makedirs(drowsy_dir, exist_ok=True)
    os.makedirs(awake_dir, exist_ok=True)

    # Load the video
    cap = cv2.VideoCapture(video_path)

    # Get total frame count of the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"\n{scenario_video_name} | frames: {total_frames} | labels: {len(labels)}")

    # Frame extraction rate (step)
    step = 15 if scenario in ["night_noglasses", "nightglasses"] else 30

    frame_num = 0
    while cap.isOpened():
        success, frame = cap.read()  # Read one frame

        # Stop if video ends or labels are finished
        if not success or frame_num >= len(labels):
            break

        # Keep only every "step" frame
        if frame_num % step != 0:
            frame_num += 1
            continue

        # Get class label from labels list
        class_id = int(labels[frame_num])  
        # class_id = 1 → drowsy
        # class_id = 0 → awake
        class_name = "awake" if class_id == 0 else "drowsy"

        # Generate unique frame name
        frame_name = f"{person_id}_{scenario_video_name}_frame{frame_num:04d}.jpg"
        save_path = os.path.join(video_output_dir, class_name, frame_name)

        # Save frame as JPG
        cv2.imwrite(save_path, frame)

        frame_num += 1

    # Close video
    cap.release()

print("all frames have been extracted")


### Testing data

In [ ]:
import os
import cv2

# Path to testing_dataset
base_input_dir = r"C:\Users\ToP NeT\Desktop\nthu_ddd\testing_dataset"
  
# Output directory where extracted frames will be saved
output_dir = r"C:\Users\ToP NeT\Desktop\nthu_ddd\nthu_ddd1"

# Collect all video files
video_files = []
for person_id in os.listdir(base_input_dir):
    person_path = os.path.join(base_input_dir, person_id)
    if not os.path.isdir(person_path):
        continue

    for file in os.listdir(person_path):
        if file.endswith(".mp4"):   # Only MP4 videos
            video_files.append(os.path.join(person_path, file))

print(f"total videos found: {len(video_files)}")

# Start extracting frames
for video_path in video_files:
    path_parts = video_path.split("\\")  # Split for Windows paths
    person_id = path_parts[-2]  # Person ID
    video_filename = os.path.splitext(path_parts[-1])[0]   # Video filename without extension

    # Path to labels file
    label_txt_path = os.path.join(base_input_dir, person_id, f"{video_filename}ing_drowsiness.txt")

    if not os.path.exists(label_txt_path):
        print(f"no labels file: {label_txt_path}")
        continue

    # Load labels
    with open(label_txt_path, "r") as f:
        labels = list(f.read().strip())

    # Output folder containing awake / drowsy subfolders
    video_output_dir = os.path.join(output_dir, person_id, video_filename)
    drowsy_dir = os.path.join(video_output_dir, "drowsy")
    awake_dir = os.path.join(video_output_dir, "awake")
    os.makedirs(drowsy_dir, exist_ok=True)
    os.makedirs(awake_dir, exist_ok=True)

    # Read video
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"\n{video_filename} | frames: {total_frames} | labels: {len(labels)}")

    step = 60  # Take one frame every 60 frames

    frame_num = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success or frame_num >= len(labels):
            break

        # Save only every 'step' frame
        if frame_num % step == 0:
            class_id = int(labels[frame_num])
            class_name = "awake" if class_id == 0 else "drowsy"

            frame_name = f"{video_filename}_frame{frame_num:04d}.jpg"
            save_path = os.path.join(video_output_dir, class_name, frame_name)

            cv2.imwrite(save_path, frame)

        frame_num += 1

    cap.release()

print("all frames have been extracted")


### Recorded data

In [ ]:
import cv2
import os
from glob import glob

# Input and output directories
input_dir = r"C:\Users\lenovo\Desktop\recorded_videos"
output_dir = r"C:\Users\lenovo\Desktop\ExtractedFrames"

# Get all .mp4 video files 
video_files = glob(os.path.join(input_dir, "*.mp4"))

for video_path in video_files:
    # Extract video file name without extension
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # Create output folder for this video's frames
    video_output_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count, saved_count = 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Rotate frame by 90 degrees counterclockwise
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        # Save 1 of every 3 frames
        if frame_count % 3 == 0:
            frame_filename = os.path.join(video_output_dir, f"frame_{saved_count:04d}.jpg")

            # Save frame as JPG
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()

print("All videos processed successfully")
